<a href="https://colab.research.google.com/github/Andiko-K/SkinColorDetection/blob/main/transfer-learning/model/skinage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import zipfile
import os, shutil

import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub

import random
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from PIL import Image

# Installing Dataset

In [ ]:
!gdown --id 1DykQEw5JvLudKa2NIRQDuwo-IzB3Bntj

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1DykQEw5JvLudKa2NIRQDuwo-IzB3Bntj
From (redirected): https://drive.google.com/uc?id=1DykQEw5JvLudKa2NIRQDuwo-IzB3Bntj&confirm=t&uuid=5b67eb40-c75f-4290-9e0a-84defbbdf6f1
To: /content/face_age.zip
100% 222M/222M [00:08<00:00, 25.6MB/s]


# Data Exploration

In [ ]:
local_file = 'face_age.zip'
destination = './face_age/'

zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall(destination)
zip_ref.close()

# Data Splitting

In [ ]:
original_dir = './face_age'
X = []
Y = []
for count, (folder_name, _, filenames) in enumerate(os.walk(original_dir)):
  if count != 0:
    for file_ in filenames:
      file_path = os.path.join(folder_name, file_)
      image = Image.open(file_path)
      image = image.convert('RGB')
      image = image.resize((128, 128))
      X.append(np.array(image))
      Y.append(int(folder_name[-3:]))

X = np.array(X)/255.0
X = X.astype('float32')
Y = np.array(Y)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y,
    train_size=.8,
    shuffle=True,
    stratify=Y
)

# Tensorflow Hub Model

In [ ]:
transfer_model = ('https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/035-128-feature-vector/2', 128)

# Define Model

In [ ]:
def create_model(transfer_model = None):
  module_handle, shape = transfer_model
  image_size = (shape, shape)
  feature_extractor = hub.KerasLayer(module_handle, input_shape = image_size + (3,), trainable = True)

  model = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(.5),
      tf.keras.layers.Dense(1)
  ])

  optimizer = 'adam'
  model.compile(optimizer = optimizer,
                loss = 'mae',
                metrics = ['mae'])
  return model

# Train Model

In [ ]:
model = create_model(transfer_model = transfer_model)

In [ ]:
EarlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'val_mae', patience = 10, mode = 'min',
                                             restore_best_weights = True, start_from_epoch = 10,
                                             min_delta = .001)
ReduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_mae', mode = 'min', min_delta = .001, patience = 3)

model.fit(X_train, Y_train, epochs = 100, callbacks = [ReduceLR, EarlyStop],
          validation_data = (X_test, Y_test))

Epoch 1/100
125/125 [==============================] - 46s 137ms/step - loss: 30.4135 - mae: 30.3817 - val_loss: 27.0903 - val_mae: 27.0581 - lr: 0.0010
Epoch 2/100
125/125 [==============================] - 13s 107ms/step - loss: 12.6187 - mae: 12.5856 - val_loss: 15.3545 - val_mae: 15.3206 - lr: 0.0010
Epoch 3/100
125/125 [==============================] - 12s 95ms/step - loss: 8.3237 - mae: 8.2894 - val_loss: 8.1294 - val_mae: 8.0948 - lr: 0.0010
Epoch 4/100
125/125 [==============================] - 14s 114ms/step - loss: 7.2978 - mae: 7.2630 - val_loss: 8.0645 - val_mae: 8.0295 - lr: 0.0010
Epoch 5/100
125/125 [==============================] - 11s 86ms/step - loss: 6.8627 - mae: 6.8275 - val_loss: 7.0162 - val_mae: 6.9807 - lr: 0.0010
Epoch 6/100
125/125 [==============================] - 11s 90ms/step - loss: 6.5606 - mae: 6.5249 - val_loss: 7.3961 - val_mae: 7.3603 - lr: 0.0010
Epoch 7/100
125/125 [==============================] - 12s 98ms/step - loss: 6.2872 - mae: 6.2512 - v

# Save Model

In [ ]:
model_path = './model/skinage_mobilev2.h5'
model.save(model_path)

In [ ]:
from google.colab import files
files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>